# Loan Data from Prosper
## by Marc Campmany

## Preliminary Wrangling

> This data set contains 113,937 loans with 81 variables on each loan, including loan amount, borrower rate (or interest rate), current loan status, borrower income, and many others.

Some questions to take into account:
-     What factors affect a loan’s outcome status?
-     What affects the borrower’s APR or interest rate?
-     Are there differences between loans depending on how large the original loan amount was?

In [99]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import os
import datetime

%matplotlib inline

> Load in your dataset and describe its properties through the questions below.
Try and motivate your exploration goals through this section.

In [87]:
df = pd.read_csv('prosperLoanData.csv')

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 81 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ListingKey                           113937 non-null  object 
 1   ListingNumber                        113937 non-null  int64  
 2   ListingCreationDate                  113937 non-null  object 
 3   CreditGrade                          28953 non-null   object 
 4   Term                                 113937 non-null  int64  
 5   LoanStatus                           113937 non-null  object 
 6   ClosedDate                           55089 non-null   object 
 7   BorrowerAPR                          113912 non-null  float64
 8   BorrowerRate                         113937 non-null  float64
 9   LenderYield                          113937 non-null  float64
 10  EstimatedEffectiveYield              84853 non-null   float64
 11  EstimatedLoss

**As the number of columns to work with is too large, we will trim down the variables to the ones found interesting for the analysis. These will be the following:**

| Variable | Description |
| --- | --- |
|	ListingNumber	|	The number that uniquely identifies the listing to the public as displayed on the website.	|
|	LoanOriginationDate | The date the loan was originated.	|
|	Term	|	The length of the loan expressed in months.	|
|	LoanStatus	|	The current status of the loan: Cancelled,  Chargedoff, Completed, Current, Defaulted, FinalPaymentInProgress, PastDue. The PastDue status will be accompanied by a delinquency bucket.	|
|	BorrowerAPR	|	The Borrower's Annual Percentage Rate (APR) for the loan.	|
|	BorrowerRate	|	The Borrower's interest rate for this loan. 	|
|	LenderYield	|	The Lender yield on the loan. Lender yield is equal to the interest rate on the loan less the servicing fee.	|
|	ProsperRating (Alpha)	|	The Prosper Rating assigned at the time the listing was created between AA - HR.  Applicable for loans originated after July 2009. 0 - N/A, 1 - HR, 2 - E, 3 - D, 4 - C, 5 - B, 6 - A, 7 - AA.|
|	ProsperScore	|	A custom risk score built using historical Prosper data. The score ranges from 1-10, with 10 being the best, or lowest risk score.  Applicable for loans originated after July 2009.	|
|	ListingCategory	|	The category of the listing that the borrower selected when posting their listing: 0 - Not Available, 1 - Debt Consolidation, 2 - Home Improvement, 3 - Business, 4 - Personal Loan, 5 - Student Use, 6 - Auto, 7- Other, 8 - Baby&Adoption, 9 - Boat, 10 - Cosmetic Procedure, 11 - Engagement Ring, 12 - Green Loans, 13 - Household Expenses, 14 - Large Purchases, 15 - Medical/Dental, 16 - Motorcycle, 17 - RV, 18 - Taxes, 19 - Vacation, 20 - Wedding Loans	|
|	BorrowerState	|	The two letter abbreviation of the state of the address of the borrower at the time the Listing was created.	|
|	Occupation	|	The Occupation selected by the Borrower at the time they created the listing.	|
|	EmploymentStatus	|	The employment status of the borrower at the time they posted the listing.	|
|	IsBorrowerHomeowner	|	A Borrower will be classified as a homowner if they have a mortgage on their credit profile or provide documentation confirming they are a homeowner.	|
|	CurrentCreditLines	|	Number of current credit lines at the time the credit profile was pulled.	|
|	AvailableBankcardCredit	|	The total available credit via bank card at the time the credit profile was pulled.	|
|	DebtToIncomeRatio	|	The debt to income ratio of the borrower at the time the credit profile was pulled. This value is Null if the debt to income ratio is not available. This value is capped at 10.01 (any debt to income ratio larger than 1000% will be returned as 1001%).	|
|	IncomeRange	|	The income range of the borrower at the time the listing was created.	|
|	LoanOriginalAmount	|	The origination amount of the loan.	|

In [94]:
columns = ['ListingNumber','LoanOriginationDate','Term', 'LoanStatus','BorrowerAPR','BorrowerRate','LenderYield','ProsperRating (Alpha)','ProsperScore','ListingCategory (numeric)','BorrowerState', 
           'Occupation','EmploymentStatus','IsBorrowerHomeowner','CurrentCreditLines','AvailableBankcardCredit','DebtToIncomeRatio','IncomeRange','LoanOriginalAmount']
loans = df[columns]
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 19 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ListingNumber              113937 non-null  int64  
 1   LoanOriginationDate        113937 non-null  object 
 2   Term                       113937 non-null  int64  
 3   LoanStatus                 113937 non-null  object 
 4   BorrowerAPR                113912 non-null  float64
 5   BorrowerRate               113937 non-null  float64
 6   LenderYield                113937 non-null  float64
 7   ProsperRating (Alpha)      84853 non-null   object 
 8   ProsperScore               84853 non-null   float64
 9   ListingCategory (numeric)  113937 non-null  int64  
 10  BorrowerState              108422 non-null  object 
 11  Occupation                 110349 non-null  object 
 12  EmploymentStatus           111682 non-null  object 
 13  IsBorrowerHomeowner        11

In [95]:
loans.rename(columns={'ProsperRating (Alpha)': 'ProsperRating',"ListingCategory (numeric)":"ListingCategory"},inplace = True)

In [96]:
loans['LoanOriginationDate']= pd.to_datetime(loans['LoanOriginationDate'])

<ipython-input-96-09afb118ddcb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans['LoanOriginationDate']= pd.to_datetime(loans['LoanOriginationDate'])


In [97]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 19 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   ListingNumber            113937 non-null  int64         
 1   LoanOriginationDate      113937 non-null  datetime64[ns]
 2   Term                     113937 non-null  int64         
 3   LoanStatus               113937 non-null  object        
 4   BorrowerAPR              113912 non-null  float64       
 5   BorrowerRate             113937 non-null  float64       
 6   LenderYield              113937 non-null  float64       
 7   ProsperRating            84853 non-null   object        
 8   ProsperScore             84853 non-null   float64       
 9   ListingCategory          113937 non-null  int64         
 10  BorrowerState            108422 non-null  object        
 11  Occupation               110349 non-null  object        
 12  EmploymentStatus

**As the information for ProsperRatings and ProsperScore is only available for post July 2009, we'll just consider post July 2009 loans for the study.**

**Analyzing notnull values for both periods we can see that previous to July 2009 there are only 9 values.**

In [123]:
loans[loans.LoanOriginationDate.dt.date > datetime.date(2009, 7, 30)].ProsperRating.notnull().sum(),loans[loans.LoanOriginationDate.dt.date < datetime.date(2009, 7, 30)].ProsperRating.notnull().sum()

(84839, 9)

In [124]:
loans = loans[loans.LoanOriginationDate.dt.date > datetime.date(2009, 7, 30)]

In [42]:
# Splitting ListingCreationDate column into year,month,day,time
loans['year']= loans['ListingCreationDate'].apply(lambda x: x.split("-")[0]).astype(str)
loans['month'] = loans['ListingCreationDate'].apply(lambda x: x.split("-")[1]).astype(str)
loans.month.replace(['01','02','03','04','05','06','07','08','09','10','11','12'],['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec'],inplace = True)

loans.month.unique()

<ipython-input-42-279a95705d65>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans['year']= loans['ListingCreationDate'].apply(lambda x: x.split("-")[0]).astype(str)
<ipython-input-42-279a95705d65>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans['month'] = loans['ListingCreationDate'].apply(lambda x: x.split("-")[1]).astype(str)


IndexError: list index out of range

In [45]:
loans['ListingCreationDate'] = loans['ListingCreationDate'].apply(lambda x: x.split("-")[2]).astype(str)

IndexError: list index out of range

In [47]:
loans['day'] = loans['ListingCreationDate'].apply(lambda x: x.split(" ")[0]).astype(str)
loans['time'] = loans['ListingCreationDate'].apply(lambda x: x.split(" ")[1]).astype(str)

<ipython-input-47-48b91cf06fc8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans['day'] = loans['ListingCreationDate'].apply(lambda x: x.split(" ")[0]).astype(str)
<ipython-input-47-48b91cf06fc8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans['time'] = loans['ListingCreationDate'].apply(lambda x: x.split(" ")[1]).astype(str)


In [48]:
loans.head()

,ListingNumber,ListingCreationDate,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,ProsperRating (Alpha),ProsperScore,...,IsBorrowerHomeowner,CurrentCreditLines,AvailableBankcardCredit,DebtToIncomeRatio,IncomeRange,LoanOriginalAmount,year,month,day,time
0,193129,26 19:09:29.263000000,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,NaN,NaN,...,True,5.0,1500.0,0.17,"$25,000-49,999",9425,2007,08,26,19:09:29.263000000
1,1209647,27 08:28:07.900000000,36,Current,NaN,0.12016,0.0920,0.0820,A,7.0,...,False,14.0,10266.0,0.18,"$50,000-74,999",10000,2014,02,27,08:28:07.900000000
2,81716,05 15:00:47.090000000,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,NaN,NaN,...,False,NaN,NaN,0.06,Not displayed,3001,2007,01,05,15:00:47.090000000
3,658116,22 11:02:35.010000000,36,Current,NaN,0.12528,0.0974,0.0874,A,9.0,...,True,5.0,30754.0,0.15,"$25,000-49,999",10000,2012,10,22,11:02:35.010000000
4,909464,14 18:38:39.097000000,36,Current,NaN,0.24614,0.2085,0.1985,D,4.0,...,True,19.0,695.0,0.26,"$100,000+",15000,2013,09,14,18:38:39.097000000


In [ ]:
loans['day'] = loans['ListingCreationDate'].apply(lambda x: x.split(" ")[2]).astype(str)
loans['time'] = loans['ListingCreationDate'].apply(lambda x: x.split(" ")[3]).astype(str)

In [34]:
loans.head()

,ListingNumber,ListingCreationDate,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,ProsperRating (Alpha),ProsperScore,...,IsBorrowerHomeowner,CurrentCreditLines,AvailableBankcardCredit,DebtToIncomeRatio,IncomeRange,LoanOriginalAmount,year,month,day,time
0,193129,2007-08-26 19:09:29.263000000,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,NaN,NaN,...,True,5.0,1500.0,0.17,"$25,000-49,999",9425,2007,08,2007-08-26,19:09:29.263000000
1,1209647,2014-02-27 08:28:07.900000000,36,Current,NaN,0.12016,0.0920,0.0820,A,7.0,...,False,14.0,10266.0,0.18,"$50,000-74,999",10000,2014,02,2014-02-27,08:28:07.900000000
2,81716,2007-01-05 15:00:47.090000000,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,NaN,NaN,...,False,NaN,NaN,0.06,Not displayed,3001,2007,01,2007-01-05,15:00:47.090000000
3,658116,2012-10-22 11:02:35.010000000,36,Current,NaN,0.12528,0.0974,0.0874,A,9.0,...,True,5.0,30754.0,0.15,"$25,000-49,999",10000,2012,10,2012-10-22,11:02:35.010000000
4,909464,2013-09-14 18:38:39.097000000,36,Current,NaN,0.24614,0.2085,0.1985,D,4.0,...,True,19.0,695.0,0.26,"$100,000+",15000,2013,09,2013-09-14,18:38:39.097000000


### What is the structure of your dataset?

> Your answer here!

### What is/are the main feature(s) of interest in your dataset?

> Your answer here!

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> Your answer here!

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

> Make sure that, after every plot or related series of plots, that you
include a Markdown cell with comments about what you observed, and what
you plan on investigating next.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!